In [1]:
import tiktoken

In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

/Users/mmontgomery14/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terrace of the palace."
text = " <|endoftext|> ".join([text1, text2])
tokens = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(tokens)
print(tokenizer.decode(tokens))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 558, 286, 262, 20562, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terrace of the palace.


In [4]:
text = ("Akwirw ier")
tokens = tokenizer.encode(text)
print(tokens)
print(tokenizer.decode(tokens))

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [5]:
t = [tokenizer.decode([token]) for token in tokens]
t

['Ak', 'w', 'ir', 'w', ' ', 'ier']

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
tokens = tokenizer.encode(raw_text) 
print(len(tokens))

5145


In [7]:
enc_sample = tokens[50:]

In [8]:
context_size = 4
for i in range(1, context_size+1):
    context = enc_sample[:i]
    y = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([y]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [9]:
import torch

In [10]:
context = 10
stride = 1
tokenizer = tiktoken.get_encoding("gpt2")
txt = raw_text

In [12]:
x_train = []
y_train = []
enc_txt = tokenizer.encode(txt)
for i in range(0, len(enc_txt) - context, stride):
    x = enc_txt[i:i+context]
    y = enc_txt[i+1:i+context+1]
    x_train.append(torch.tensor(x))
    y_train.append(torch.tensor(y))
x_train = torch.stack(x_train)
y_train = torch.stack(y_train)
print(x_train.shape, y_train.shape)
print(x_train[:10])
print(y_train[:10])


torch.Size([5135, 10]) torch.Size([5135, 10])
tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026, 15632,   438],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632,   438,  2016],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [10899,  2138,   257,  7026, 15632,   438,  2016,   257,   922,  5891],
        [ 2138,   257,  7026, 15632,   438,  2016,   257,   922,  5891,  1576]])
tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  

In [15]:
dataset = GPTDatasetV1(raw_text, tiktoken.get_encoding("gpt2"), context_size=10)

In [22]:
print(f"Dataset length: {len(dataset)}")
x, y = dataset[:]
print(f"Input shape: {x.shape}, Target shape: {y.shape}")

Dataset length: 5135
Input shape: torch.Size([5135, 10]), Target shape: torch.Size([5135, 10])


In [23]:
dataloader = dataset.load_data()
batch_count = 0
for x, y in dataloader:
    print(f"Batch {batch_count}:")
    print(f"Input shape: {x.shape}, Target shape: {y.shape}")
    batch_count += 1
    if batch_count == 3:  # Just showing first 3 batches as example
        break

print(f"\nTotal number of batches: {len(dataloader)}")

Batch 0:
Input shape: torch.Size([32, 10]), Target shape: torch.Size([32, 10])
Batch 1:
Input shape: torch.Size([32, 10]), Target shape: torch.Size([32, 10])
Batch 2:
Input shape: torch.Size([32, 10]), Target shape: torch.Size([32, 10])

Total number of batches: 160


In [26]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(f"Input shape: {first_batch[0].shape}, Target shape: {first_batch[1].shape}")



Input shape: torch.Size([32, 10]), Target shape: torch.Size([32, 10])


In [27]:
dataset = GPTDatasetV1(raw_text, tiktoken.get_encoding("gpt2"), context_size=8, stride=2)
dataloader = dataset.load_data(batch_size=10)
data_iter = iter(dataloader)
x, y = next(data_iter)
print("Batch Example:")
print("Inputs: ", x.shape, "\n", x)
print("Targets: ", y.shape, "\n", y)
print("Number of batches:", len(dataloader))



Batch Example:
Inputs:  torch.Size([10, 8]) 
 tensor([[  198,   198,  1890,   262,   717,   640,   616, 21696],
        [  438,   261, 45697, 19369,    11,   355,   345,   910],
        [  284,   467,   257,  1310,  4295,   438,    40,  2936],
        [  257, 17779,   339,   550, 11564,   284,  1657,    13],
        [  339,   531, 15376,    26,   788,    11,  6427,   465],
        [  743,   307, 41746, 12004,   262,  6473,   438,  5562],
        [11428,   450,    67,  3299,    13,   887,   645,   438],
        [   40,  1297,  9074,    13,   520,  5493,   523,   618],
        [10724,   262,  6846,   338, 11428,   450,    67,  3299],
        [   13,  3226,  1781,   314,  1422,   470,  1560,   607]])
Targets:  torch.Size([10, 8]) 
 tensor([[  198,  1890,   262,   717,   640,   616, 21696, 20136],
        [  261, 45697, 19369,    11,   355,   345,   910,    13],
        [  467,   257,  1310,  4295,   438,    40,  2936, 10927],
        [17779,   339,   550, 11564,   284,  1657,    13, 24975

In [28]:
import torch.nn as nn

In [29]:
vocab_size = tokenizer.n_vocab
embedding_dim = 32
embedding_layer = nn.Embedding(vocab_size, embedding_dim)
print(embedding_layer.weight.shape)

torch.Size([50257, 32])


In [30]:
# Get a batch of data
data_iter = iter(dataloader)
x, _ = next(data_iter)

# Embed the batch
embedded = embedding_layer(x)
print("Input shape:", x.shape)
print("Embedded shape:", embedded.shape)

Input shape: torch.Size([10, 8])
Embedded shape: torch.Size([10, 8, 32])


In [31]:
print(embedded)

tensor([[[ 1.8631e+00,  8.4178e-01,  9.7228e-01,  ..., -9.6654e-01,
           3.2825e-02, -9.1771e-01],
         [-6.6487e-01,  1.2974e+00,  2.1441e-01,  ..., -5.7817e-01,
          -1.1918e+00,  1.9304e-01],
         [ 7.2200e-01,  1.7812e+00,  1.5288e+00,  ...,  1.4561e-01,
           1.4557e+00, -8.1448e-01],
         ...,
         [-9.7156e-03,  1.1120e+00, -5.2111e-01,  ...,  1.6579e+00,
           2.5478e-01, -9.6888e-01],
         [ 6.7424e-01,  5.6093e-01, -8.5805e-01,  ...,  1.1551e-01,
          -1.0838e+00, -5.2731e-01],
         [ 1.8631e+00,  8.4178e-01,  9.7228e-01,  ..., -9.6654e-01,
           3.2825e-02, -9.1771e-01]],

        [[-1.7048e-01,  2.0849e-01,  7.1294e-01,  ..., -4.4171e-02,
          -1.2940e+00,  2.9590e-01],
         [-5.6068e-01, -4.5873e-01, -4.6548e-01,  ...,  2.1205e-03,
           1.4232e+00,  1.8232e+00],
         [ 2.4071e-01,  9.0306e-01, -1.7855e+00,  ..., -1.0483e+00,
           2.2495e-01,  9.9595e-01],
         ...,
         [ 2.1060e+00,  1

In [36]:
print(embedded[0][0])
print(embedded[0][0].shape)

tensor([ 1.8631,  0.8418,  0.9723, -0.5850,  0.3627, -0.2022,  0.3914, -0.1290,
         0.8355,  1.2098,  1.6261, -0.4519, -0.0081,  0.9334,  2.2611,  0.0725,
         0.4023, -0.1037, -0.5099, -0.8252,  0.0823,  0.5279, -0.9744,  0.0575,
        -1.2271,  1.2841,  0.9220,  0.3859,  0.5539, -0.9665,  0.0328, -0.9177],
       grad_fn=<SelectBackward0>)
torch.Size([32])


In [14]:
from torch.utils.data import DataLoader, Dataset

# dataset and dataloader
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, context_size=4, stride=1):
        self.x = []
        self.y = []
        enc_txt = tokenizer.encode(text)
        for i in range(0, len(enc_txt) - context_size, stride):
            x = enc_txt[i:i+context_size]
            y = enc_txt[i+1:i+context_size+1]
            self.x.append(torch.tensor(x))
            self.y.append(torch.tensor(y))
        self.x = torch.stack(self.x)
        self.y = torch.stack(self.y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def load_data(self, batch_size=32, shuffle=True):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, drop_last=True, num_workers=0)


In [51]:
# parameters
tokenizer = tiktoken.get_encoding("gpt2")
vocab_size = tokenizer.n_vocab
context_size = 4
embedding_dim = 256
# layers
token_embedding_layer = nn.Embedding(vocab_size, embedding_dim) 
positional_embedding_layer = nn.Embedding(context_size, embedding_dim)

In [53]:
# dataset
dataset = GPTDatasetV1(raw_text, tokenizer, context_size=context_size, stride=1)
# dataloader
dataloader = dataset.load_data(batch_size=8)
data_iter = iter(dataloader)

In [56]:
# batch
x, y = next(data_iter)
# embed batch
token_embedded = token_embedding_layer(x) # [B, T, C]
positional_embedded = positional_embedding_layer(torch.arange(context_size)) # [T, C]
embedded = token_embedded + positional_embedded # [B, T, C]
# TODO: review broadcasting